# 📊 Case Study 2: Statistical Enhancements for 90-Day Inhalation Toxicity Study

This notebook provides low-to-moderate cost enhancements to the original analysis of the 90-day inhalation toxicity study evaluating PG/VG with and without nicotine in rats. It includes:

- ANCOVA for adjusted comparisons
- Mixed-effects models for repeated measurements
- Effect size estimation (Cohen’s d)
- PCA for dimensionality reduction
- Sensitivity analysis
- Power analysis

The enhancements are aligned with regulatory expectations and aim to improve precision, interpretability, and reproducibility.


## 📁 Simulate Example Dataset

In [ ]:

import pandas as pd
import numpy as np

np.random.seed(42)
groups = ["Sham", "Saline", "PGVG_Low", "PGVG_Med", "PGVG_High", "Nic_Low", "Nic_Med", "Nic_High"]
n_animals = 10

data = []
for group in groups:
    for i in range(n_animals):
        animal_id = f"{group}_{i}"
        baseline_weight = np.random.normal(250, 10)
        timepoints = [0, 30, 60, 90]
        for t in timepoints:
            weight = baseline_weight + np.random.normal(0.5*t, 3)
            lung_weight = np.random.normal(1.5 + 0.1*groups.index(group), 0.1)
            data.append([animal_id, group, t, baseline_weight, weight, lung_weight])

df = pd.DataFrame(data, columns=["animal_id", "group", "timepoint", "baseline_weight", "weight", "lung_weight"])
df.head()


## 📐 ANCOVA: Adjusting Lung Weight for Baseline Body Weight


In this study, different exposure groups might begin with slightly different body weights. Using ANCOVA allows us to compare lung weights between groups while adjusting for these baseline differences, improving the precision of our inference.


In [ ]:

import statsmodels.formula.api as smf

# Use final timepoint only for ANCOVA
ancova_data = df[df["timepoint"] == 90]
model = smf.ols("lung_weight ~ C(group) + baseline_weight", data=ancova_data).fit()
print(model.summary())


## 🔄 Mixed Effects Model: Longitudinal Weight Changes


To properly model repeated measurements of weight over time for each rat, a linear mixed-effects model accounts for both group effects and within-animal correlation. This is critical in toxicology studies where weight gain or loss over time is monitored.


In [ ]:

import statsmodels.api as sm

md = smf.mixedlm("weight ~ C(group) * timepoint", df, groups=df["animal_id"])
mdf = md.fit()
print(mdf.summary())


## 📏 Cohen’s d: Effect Size Between Sham and PGVG High


Effect size provides an estimate of the magnitude of difference between two groups. In regulatory toxicology, it's important to know not just whether a change is statistically significant, but whether it is biologically meaningful.


In [ ]:

control = ancova_data[ancova_data["group"] == "Sham"]["lung_weight"]
high_dose = ancova_data[ancova_data["group"] == "PGVG_High"]["lung_weight"]
cohen_d = (high_dose.mean() - control.mean()) / np.sqrt((control.var() + high_dose.var()) / 2)
print(f"Cohen's d: {cohen_d:.3f}")


## 🔍 PCA: Organ Weight Dimensionality Reduction


Principal Component Analysis helps visualize clustering and identify group separation in multivariate data. This is helpful when analyzing organ weights or omics data to identify patterns related to exposure.


In [ ]:

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

pca_data = ancova_data[["lung_weight", "baseline_weight"]]
pca_std = (pca_data - pca_data.mean()) / pca_data.std()
pca = PCA(n_components=2)
pca_out = pca.fit_transform(pca_std)

ancova_data["PC1"] = pca_out[:, 0]
ancova_data["PC2"] = pca_out[:, 1]

plt.figure(figsize=(8, 6))
sns.scatterplot(data=ancova_data, x="PC1", y="PC2", hue="group")
plt.title("PCA of Organ and Baseline Weight")
plt.show()


## 🧪 Sensitivity Analysis: Excluding PGVG_High Group


Sensitivity analysis checks the robustness of our findings. Here, we re-run the ANCOVA excluding the high dose group to see how much it influences the results.


In [ ]:

sens_data = ancova_data[ancova_data["group"] != "PGVG_High"]
sens_model = smf.ols("lung_weight ~ C(group) + baseline_weight", data=sens_data).fit()
print(sens_model.summary())


## ⚡ Power Analysis: Sample Size Planning


Power analysis is used to determine whether the sample size is sufficient to detect a biologically relevant effect. This is especially important in transcriptomics or when null results are observed.


In [ ]:

from statsmodels.stats.power import TTestIndPower

# Estimate power for effect size observed between Sham and PGVG_High
analysis = TTestIndPower()
power = analysis.power(effect_size=cohen_d, nobs1=10, alpha=0.05, ratio=1.0)
print(f"Estimated power: {power:.2f}")
